# Environment  
## Load required packages  


In [ ]:
library(Seurat)
library(DoubletFinder)
library(tidyverse)
library(viridis)

In [ ]:
sample <- params$sample # sample_condition

## Loading data  
*min.cells* : Include features detected in at least this many cells. Will subset the counts matrix as well.  
*min.features* : Include cells where at least this many features are detected.  


In [ ]:
seurat_obj <- readRDS(file = paste0("./results/rds/samples/Doublet_", sample, ".rds"))

## Raw data
QC_sample <- readRDS(file = paste0("./results/rds/samples/QC_", sample, ".rds"))

seurat_obj                  


##  <span style="color:#296d98;"> Parameters </span>  


In [ ]:
pc = as.numeric(params$pc)
doublet_rate = as.double(params$doublet_rate)

# Doublet
### Pre-process standard workflow  


In [ ]:
seurat_obj <- FindNeighbors(seurat_obj, dims = 1:pc)
seurat_obj <- FindClusters(seurat_obj)
seurat_obj <- RunUMAP(seurat_obj, dims = 1:pc)


### pk optimisation  


In [ ]:
sweep.res.list <- paramSweep_v3(seurat_obj, PCs = 1:pc, sct = FALSE)

In [ ]:
sweep.stats <- summarizeSweep(sweep.res.list, GT=FALSE)
bcmvn <- find.pK(sweep.stats)

In [ ]:
ggplot(bcmvn, aes(pK, BCmetric, group=1)) +
      geom_point() +
      geom_line()

# we select the pK that corresponds to max bcmvn to optimize doublet detection 
pK <- bcmvn %>%
  filter(BCmetric == max(BCmetric)) %>%
  select(pK) 
pK <- as.numeric(as.character(pK[[1]]))
pK


### Homotypic Doublet Proportion Estimate 


In [ ]:
annotations <- seurat_obj@meta.data$seurat_clusters
homotypic.prop <- modelHomotypic(annotations)
nExp_poi <- round(doublet_rate*nrow(seurat_obj@meta.data))  
nExp_poi.adj <- round(nExp_poi*(1-homotypic.prop))

# run doubletFinder 
seurat_obj <- doubletFinder_v3(seurat_obj, 
                                     PCs = 1:pc, 
                                     pN = 0.25, 
                                     pK = pK, 
                                     nExp = nExp_poi.adj,
                                     reuse.pANN = FALSE, sct = FALSE)


### Singlets and doublets  


In [ ]:
## retrieve column which classifies Singlets/Doublets
col = colnames(seurat_obj[[]])[which(grepl('^DF.classifications', colnames(seurat_obj[[]])) ==TRUE )]

## add this classification to raw data QC_sample
QC_sample@meta.data$DF.classifications <- seurat_obj@meta.data[col][,1]


In [ ]:
# visualize doublets
DimPlot(seurat_obj, reduction = 'umap', group.by = col)

# number of singlets and doublets
table(QC_sample@meta.data$DF.classifications)


### Filter out doublets


In [ ]:
subset(
  QC_sample,
  DF.classifications == "Singlet"
  ) -> QC_sample

QC_sample


# QC metrics (Filtered)


In [ ]:
FeatureScatter(QC_sample, feature1 = "nCount_RNA", feature2 = "nFeature_RNA") + geom_smooth(method = 'lm')

In [ ]:
as_tibble(
  QC_sample[[]],
  rownames="Cell.Barcode"
) -> qc.metrics


## Nb of reads   


In [ ]:
ggplot(qc.metrics) + 
   geom_histogram(aes(nCount_RNA), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 50) + 
  ggtitle("Distribution of nCount_RNA ") + NoLegend()


## Nb of genes  


In [ ]:
ggplot(qc.metrics) + 
   geom_histogram(aes(nFeature_RNA), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 50) + 
  ggtitle("Distribution of nFeature_RNA ") + NoLegend()

## MT genes  


In [ ]:
ggplot(qc.metrics) + 
   geom_histogram(aes(percent.mt), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) + 
  scale_x_continuous(breaks=seq(0, 100, by = 15))+
  ggtitle("Distribution of Percentage Mitochondrion") + NoLegend()


## Ribosomal genes  


In [ ]:
ggplot(qc.metrics) + 
   geom_histogram(aes(percent.ribosomal), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) + 
  ggtitle("Distribution of Percentage Ribosomal") + NoLegend()


## Largest gene  


In [ ]:
ggplot(qc.metrics) + 
   geom_histogram(aes(percent.largest_gene), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Largest Gene") + NoLegend()

 
# Save seurat object  


In [ ]:
saveRDS(QC_sample, file =paste0("./results/rds/samples/Filtered_",sample,".rds"))